In [1]:
import mint
import numpy

In [12]:
regridder = mint.RegridEdges()

# grid settings
regridder.setSrcGridFlags(fixLonAcrossDateline=0, averageLonAtPole=0)
regridder.setDstGridFlags(fixLonAcrossDateline=1, averageLonAtPole=1)

# load the grids
regridder.loadSrcGrid('../../data/latlon100x50.nc$latlon')
regridder.loadDstGrid('../../data/lfric_diag_wind.nc$Mesh2d')

# compute the regridding weights
regridder.build(numCellsPerBucket=1, periodX=360., debug=1)

mint.writeLogMessages('log.txt')

In [7]:
dst_grid = mint.Grid()
# settings for the cubed-sphere grid
dst_grid.setFlags(fixLonAcrossDateline=1, averageLonAtPole=1)
dst_grid.loadFromUgrid2D('../../data/lfric_diag_wind$Mesh2d')

ERROR:root:ier=1 after calling loadFromUgrid2D in grid.py!


RuntimeError: ier=1 after calling loadFromUgrid2D in grid.py!